In [ ]:
import gr00t
from gr00t.policy.gr00t_policy import Gr00tPolicy
from custom_modality_config import *
from gr00t.data.types import EmbodimentTag

from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata

import torch
from transformers import AutoModel, AutoProcessor

from IPython.display import clear_output


/workspace/gr00t/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'lerobot'

In [ ]:
repo_id = "LSY-lab/stack_without_ft_tact_v4"
ds_metadata = LeRobotDatasetMetadata(repo_id)
dataset = LeRobotDataset(repo_id, episodes=[0, 10, 11, 23])

In [ ]:
import gr00t.model

model = AutoModel.from_pretrained("nvidia/GR00T-N1.6-3B")
model.to(device="cuda", dtype=torch.bfloat16)
clear_output()

In [ ]:
processor = AutoProcessor.from_pretrained("nvidia/GR00T-N1.6-3B")

In [5]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
from transformers import AutoProcessor
import torch

repo_id = "LSY-lab/stack_without_ft_tact_v4"
episodes = [0, 10, 11, 23]

# Load dataset and metadata
metadata = LeRobotDatasetMetadata(repo_id)
dataset = LeRobotDataset(repo_id, episodes=episodes)

# Load GR00T processor
processor = AutoProcessor.from_pretrained("nvidia/GR00T-N1.6-3B")

def collate_and_add_key(batch):
    # Remove 'task' key if present, and add dummy keys for model compatibility
    for b in batch:
        if 'task' in b:
            del b['task']
        # Add a dummy input_ids if missing
        if 'input_ids' not in b and 'index' in b:
            b['input_ids'] = b['index']
        # Add a dummy attention_mask if missing
        if 'attention_mask' not in b:
            # Try to infer correct shape from input_ids or set to ones
            if 'input_ids' in b and hasattr(b['input_ids'], 'shape'):
                b['attention_mask'] = torch.ones_like(b['input_ids'])
            else:
                b['attention_mask'] = torch.ones(1, dtype=torch.long)
    return batch  # Do not call processor.collator since it's not implemented

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_and_add_key
)

batch = next(iter(dataloader))
# Now batch is a list of dicts with required keys, including attention_mask


In [10]:
dataloader_no_collate = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
)
batch = next(iter(dataloader_no_collate))

In [15]:
batch.pop('task', None)  # Remove 'task' key if present

['pick the lego block.']

In [16]:
batch

{'observation.images.primary': tensor([[[[0.6667, 0.6667, 0.6667,  ..., 0.5490, 0.5647, 0.5725],
           [0.6667, 0.6667, 0.6667,  ..., 0.5569, 0.5373, 0.4941],
           [0.6667, 0.6667, 0.6667,  ..., 0.3882, 0.3765, 0.5216],
           ...,
           [0.5333, 0.5216, 0.5255,  ..., 0.7333, 0.7333, 0.7333],
           [0.5294, 0.5216, 0.5216,  ..., 0.7333, 0.7333, 0.7333],
           [0.5294, 0.5294, 0.5294,  ..., 0.7333, 0.7333, 0.7333]],
 
          [[0.6941, 0.6941, 0.6941,  ..., 0.5804, 0.5961, 0.6039],
           [0.6941, 0.6941, 0.6941,  ..., 0.5882, 0.5686, 0.5255],
           [0.6941, 0.6941, 0.6941,  ..., 0.4196, 0.4078, 0.5529],
           ...,
           [0.5529, 0.5412, 0.5451,  ..., 0.7451, 0.7451, 0.7451],
           [0.5490, 0.5412, 0.5412,  ..., 0.7451, 0.7451, 0.7451],
           [0.5490, 0.5490, 0.5490,  ..., 0.7451, 0.7451, 0.7451]],
 
          [[0.6863, 0.6863, 0.6863,  ..., 0.5333, 0.5490, 0.5569],
           [0.6863, 0.6863, 0.6863,  ..., 0.5412, 0.5216, 0.4

In [25]:
model.config.image_token_index

AttributeError: 'Gr00tN1d6Config' object has no attribute 'image_token_index'

In [23]:
backbone_inputs, action_inputs = model.prepare_input(batch)
model.backbone(backbone_inputs)

KeyError: 'input_ids'

In [18]:
model.eval()  # Set to evaluation mode
with torch.no_grad():
    model(batch)

KeyError: 'input_ids'

In [8]:
model.train()  # Set to training mode
clear_output()


In [9]:
# For inference only (no loss calculation)
model.eval()
with torch.no_grad():
    # If batch is a list of dicts, pick one sample or merge as needed
    sample = batch[0] if isinstance(batch, list) else batch
    action_outputs = model.get_action(sample)
    predicted_actions = action_outputs['action_pred']
    print(f"Predicted action shape: {predicted_actions.shape}")
    print(f"First action: {predicted_actions[0, 0, :5]}")  # First 5 dims of first action

KeyError: 'pixel_values'

In [8]:
# For inference only (no loss calculation)
model.eval()
with torch.no_grad():
    batch_for_inference = next(iter(dataloader))
    action_outputs = model.get_action(batch_for_inference)
    predicted_actions = action_outputs['action_pred']
    print(f"Predicted action shape: {predicted_actions.shape}")
    print(f"First action: {predicted_actions[0, 0, :5]}")  # First 5 dims of first action

ValueError: too many values to unpack (expected 2)

## Inference (Get Actions without Loss)

In [ ]:
# Setup optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        
        # Forward pass - loss is computed automatically inside model
        outputs = model(batch)
        loss = outputs['loss']
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item():.4f}")
        
        # Break early for testing
        if i >= 50:
            break